# 1. Imports

In [1]:
from confluent_kafka.admin import AdminClient
import logging
import json
import time
from confluent_kafka import Producer
import typing
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("producer")

config = json.load(open("config.json"))
print(json.dumps(config, indent=2))

{
  "global": {
    "kafka_bootstrap_servers": "kafka:9092",
    "kafka_topic": "test-structured-streaming",
    "kafka_consumer_group": "ss_job",
    "max_records_per_batch": 20
  }
}


In [2]:
config['global']['kafka_topic'] = 'test'

In [3]:
data = json.load(open("data.json"))
dictVals = []
#print(data[1]['name'].values())
#print(data[0].keys())
'''
maxCount = 10
counter = 0
while counter < maxCount:
        for x in dictKeys:
            #print(counter)
            print(data[counter][x])
        #print(dictVals[counter])
        counter += 1
'''
for x in range(10):
    print(str(data[x]))
    print("")
    dictVals.append(data[x])
    #print(data[x].values())
#print(dictVals)
#print(json.dumps(data, indent=2))

{'name': {'first': 'John', 'last': 'Backus'}, 'birth': '1924-12-03T05:00:00Z', 'death': '2007-03-17T04:00:00Z', 'contribs': ['Fortran', 'ALGOL', 'Backus-Naur Form', 'FP'], 'awards': [{'award': 'W.W. McDowell Award', 'year': 1967, 'by': 'IEEE Computer Society'}, {'award': 'National Medal of Science', 'year': 1975, 'by': 'National Science Foundation'}, {'award': 'Turing Award', 'year': 1977, 'by': 'ACM'}, {'award': 'Draper Prize', 'year': 1993, 'by': 'National Academy of Engineering'}]}

{'name': {'first': 'John', 'last': 'McCarthy'}, 'birth': '1927-09-04T04:00:00Z', 'death': '2011-12-24T05:00:00Z', 'contribs': ['Lisp', 'Artificial Intelligence', 'ALGOL'], 'awards': [{'award': 'Turing Award', 'year': 1971, 'by': 'ACM'}, {'award': 'Kyoto Prize', 'year': 1988, 'by': 'Inamori Foundation'}, {'award': 'National Medal of Science', 'year': 1990, 'by': 'National Science Foundation'}]}

{'name': {'first': 'Grace', 'last': 'Hopper'}, 'title': 'Rear Admiral', 'birth': '1906-12-09T05:00:00Z', 'death

In [4]:
print(len(dictVals))

10


# 2. Data Generator

In [5]:
#print(config['global'])
def data_generator(dictVals) -> str:
    # Read data from json (data.json)
    # Push 1 record at a time - Make use of python 'yield' operator
    maxVal = 9
    counter = 0    
    #Create 10 messages based on extracted values from the dictionary
    while counter <= maxVal:
        yield str(dictVals[counter])
        counter += 1

# Push data to kafka -- data is a generator object
data = data_generator(dictVals)
#print(data)
#[next(data) for i in range(10)]
#print(next(data))

In [6]:
def data_generator() -> str:
    # Read data from json (data.json)
    # Push 1 record at a time - Make use of python 'yield' operator
    maxVal = 9
    counter = 0    
    #Create 10 messages based on extracted values from the dictionary
    while counter <= maxVal:
        yield str(data)
        counter += 1

# Push data to kafka -- data is a generator object
#data = data_generator()

# 3. Build Kafka Producer class

In [7]:
broker_conf = {
    'bootstrap.servers': config["global"]["kafka_bootstrap_servers"],
}

producer_conf = {**broker_conf}

class KafkaProducer(object):
    def __init__(self, broker_conf, debug=False):
        self.broker_conf = broker_conf
        self.producer = Producer(self.broker_conf)
        self.debug = debug

    def delivery_report(self, err, msg):
        """ Called once for each message produced to indicate delivery result.
            Triggered by poll() or flush(). """
        if err is not None:
            logger.error('Message delivery failed: {}'.format(err))
        else:
            # if self.debug:
            logger.info('Message delivered to topic: {} [parition={}]'.format(msg.topic(), msg.partition()))

    def push(self, data: typing.List[str], topic: str, delay_in_secs: int = 0):
        start_time = time.time()
        num_records = 0
        #extract messages from data generator object and store
        messages = []
        #Create producer instance -- Topic = test-structured-streaming
        #p = Producer({'bootstrap.servers': 'kafka:9092'})
        print(data)
        #Extract messages from data generator object
        #Set the number of records equal to the number of messages
        self.producer.poll(0)
        for row in data:
            print('This data was sent! ', row)
            print("")
            #Topic, data encoding, callback report
            self.producer.produce('test', row.encode('utf-8'), callback = self.delivery_report)
            num_records += 1
        self.producer.flush()
        end_time = time.time()
        completion_time = end_time-start_time
        logger.info(
            f"Pushed {num_records} records with {delay_in_secs} secs delay. Task completed in {completion_time:.2f} secs")
        return num_records, completion_time

# 4. Push data to kafka

In [8]:
kafka_produce = KafkaProducer(broker_conf=broker_conf, debug=True)
num_records, _ = kafka_produce.push(data=data,  topic='test', delay_in_secs=0)

INFO:producer:Message delivered to topic: test [parition=0]
INFO:producer:Message delivered to topic: test [parition=0]
INFO:producer:Message delivered to topic: test [parition=0]
INFO:producer:Message delivered to topic: test [parition=0]
INFO:producer:Message delivered to topic: test [parition=0]
INFO:producer:Message delivered to topic: test [parition=0]
INFO:producer:Message delivered to topic: test [parition=0]
INFO:producer:Message delivered to topic: test [parition=0]
INFO:producer:Message delivered to topic: test [parition=0]
INFO:producer:Message delivered to topic: test [parition=0]


<generator object data_generator at 0x7f64717ca6d0>
This data was sent!  {'name': {'first': 'John', 'last': 'Backus'}, 'birth': '1924-12-03T05:00:00Z', 'death': '2007-03-17T04:00:00Z', 'contribs': ['Fortran', 'ALGOL', 'Backus-Naur Form', 'FP'], 'awards': [{'award': 'W.W. McDowell Award', 'year': 1967, 'by': 'IEEE Computer Society'}, {'award': 'National Medal of Science', 'year': 1975, 'by': 'National Science Foundation'}, {'award': 'Turing Award', 'year': 1977, 'by': 'ACM'}, {'award': 'Draper Prize', 'year': 1993, 'by': 'National Academy of Engineering'}]}

This data was sent!  {'name': {'first': 'John', 'last': 'McCarthy'}, 'birth': '1927-09-04T04:00:00Z', 'death': '2011-12-24T05:00:00Z', 'contribs': ['Lisp', 'Artificial Intelligence', 'ALGOL'], 'awards': [{'award': 'Turing Award', 'year': 1971, 'by': 'ACM'}, {'award': 'Kyoto Prize', 'year': 1988, 'by': 'Inamori Foundation'}, {'award': 'National Medal of Science', 'year': 1990, 'by': 'National Science Foundation'}]}

This data was sen

INFO:producer:Pushed 10 records with 0 secs delay. Task completed in 0.04 secs


In [ ]:
from confluent_kafka import Consumer

c = Consumer({
    'bootstrap.servers': 'kafka:9092',
    'group.id': 'mygroup',
    'auto.offset.reset': 'earliest'
})

c.subscribe(['test'])

while True:
    msg = c.poll(1.0)

    if msg is None:
        continue
    if msg.error():
        print("Consumer error: {}".format(msg.error()))
        continue

    print('Received message: {}'.format(msg.value().decode('utf-8')))

c.close()

Received message: {'name': {'first': 'John', 'last': 'Backus'}, 'birth': '1924-12-03T05:00:00Z', 'death': '2007-03-17T04:00:00Z', 'contribs': ['Fortran', 'ALGOL', 'Backus-Naur Form', 'FP'], 'awards': [{'award': 'W.W. McDowell Award', 'year': 1967, 'by': 'IEEE Computer Society'}, {'award': 'National Medal of Science', 'year': 1975, 'by': 'National Science Foundation'}, {'award': 'Turing Award', 'year': 1977, 'by': 'ACM'}, {'award': 'Draper Prize', 'year': 1993, 'by': 'National Academy of Engineering'}]}
Received message: {'name': {'first': 'John', 'last': 'McCarthy'}, 'birth': '1927-09-04T04:00:00Z', 'death': '2011-12-24T05:00:00Z', 'contribs': ['Lisp', 'Artificial Intelligence', 'ALGOL'], 'awards': [{'award': 'Turing Award', 'year': 1971, 'by': 'ACM'}, {'award': 'Kyoto Prize', 'year': 1988, 'by': 'Inamori Foundation'}, {'award': 'National Medal of Science', 'year': 1990, 'by': 'National Science Foundation'}]}
Received message: {'name': {'first': 'Grace', 'last': 'Hopper'}, 'title': 'Re